# Ej 19

Cantidad de contenido por planeta fuera de la tierra en la Wikipedia.

In [1]:
    from pyspark.sql import *
    from pyspark.sql.functions import *
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    import pandas as pd

    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/31 20:12:47 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/31 20:12:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/31 20:12:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/10/31 20:12:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
geo_tags = sqlContext.read.csv('./Wikipedia_dataset_2c2021/geo_tags.csv', 
                                    header=True, inferSchema=True, multiLine=True)

geo_tags_rdd = geo_tags.rdd

## Limpio los datos

Me quedo con los registros que no tengan `None` en el campo `gt_globe`.

In [4]:
geo_tags_rdd = geo_tags_rdd.filter(lambda x: x.gt_globe != 'None')

In [5]:
# Verifico que cargue los datos correctamente.
geo_tags_rdd.take(5)

[Row(gt_id=158041, gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844377, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844381, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.70199966, gt_lon=-89.20800018, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844383, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.39999962, gt_lon=-89.75, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844388, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=12.85000038, gt_lon=-88.0, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

## Filtro los contenidos que no esten en la tierra

In [6]:
contenido_fuera_tierra = geo_tags_rdd.filter(lambda x: x.gt_globe != 'earth')

## Cuento la cantidad de contenido por planeta fuera de tierra

In [7]:
contenido_fuera_tierra.map(lambda x: (x.gt_globe,1)).reduceByKey(lambda x,y: x+y).count()

17

Dado que conozco que hay una cantidad finida de categorias, hago el collect()

In [8]:
contenido_fuera_tierra.map(lambda x: (x.gt_globe,1)).reduceByKey(lambda x,y: x+y).collect()

[('mercury', 561),
 ('mars', 1438),
 ('moon', 9061),
 ('ganymede', 116),
 ('venus', 24),
 ('umbriel', 1),
 ('titan', 4),
 ('phobos', 20),
 ('oberon', 1),
 ('callisto', 3),
 ('titania', 2),
 ('io', 2),
 ('hyperion', 1),
 ('deimos', 2),
 ('tethys', 1),
 ('mimas', 1),
 ('enceladus', 1)]

In [9]:
contenido_fuera_tierra.count()

11239

# Conclusion

Vemos que existen 11239 articulos con contenido fuera del planeta tierra en la Wikipedia, de los cuales se distribuyen en su mayoria en mercurio, marte y la luna.

Para la tabla con datos de todos los contenidos de Wikipedia en su versión más reciente que contiene `contents.csv`, observo que hay contenido para 17 planetas distintos a la tierra con contenido y su distribucion es la siguiente:

| n | Planeta | Cantidad de Contenido|
|---|---|---|
|1 | moon | 9061|
|2| mars|1438 |
|3|mercury | 561|
|4| ganymede| 116|
|5| venus| 24|
|6 |phobos |20 |
|7| titan| 4|
|8| callisto| 3|
|9| titania| 2|
|10|deimos |2 |
|11|io |2 |
|12| mimas| 1|
|13| hyperion| 1|
|14|tethys | 1|
|15|umbriel | 1|
|16|oberon | 1|
|17| enceladus| 1|

Tenemos un total de 11239 articulos con tag `gt_global` fuera del planeta tierra.